![data-x](http://oi64.tinypic.com/o858n4.jpg)

---
# Cookbook 5: SQL Example

**Author list:** Alexander Fred Ojala & Sindhuja Jeyabal

**References / Sources:** https://plot.ly/python/big-data-analytics-with-pandas-and-sqlite/

**License Agreement:** Feel free to do whatever you want with this code

___

*This notebook is an introduction on how to work with databases and query with SQL by using SQLAlchemy and Pandas in Python*

In [3]:
from __future__ import print_function, absolute_import, division

import pandas as pd
from sqlalchemy import create_engine # database connection
import datetime as dt

import warnings
warnings.filterwarnings("ignore") # filter warnings

#### Initialize database with filename 311_NYC.db in current working directory
Data can be downloaded from here: https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9 (press Download at top, then CSV)

Otherwise you can work on the small subset of the data, called: `311_NYC_small.csv`

In [ ]:
disk_engine = create_engine('sqlite:///311_NYC.db') # has already been done

In [4]:
!wc -l < 311_NYC.csv 
# count the number of rows in the data set, over 15 million records

 15001681


# Setup database from file
#### Create DataFrame and extract data to populate an SQLite local database file

In [3]:
start = dt.datetime.now()

chunksize = 20000 # how many data rows to extract at a time

j = 0
index_start = 1

for df in pd.read_csv('311_NYC.csv', chunksize=chunksize, iterator=True, encoding='utf-8'):
    
    df = df.rename(columns={c: c.replace(' ', '') for c in df.columns}) # Remove spaces from columns

    df['CreatedDate'] = pd.to_datetime(df['CreatedDate']) # Convert to datetimes
    df['ClosedDate'] = pd.to_datetime(df['ClosedDate'])

    df.index += index_start

    # Remove the un-interesting columns
    columns = ['Agency', 'CreatedDate', 'ClosedDate', 'ComplaintType', 'Descriptor',
               'CreatedDate', 'ClosedDate', 'TimeToCompletion',
               'City']

    for c in df.columns:
        if c not in columns:
            df = df.drop(c, axis=1)    

    
    j+=1
    print('{} seconds: completed {} rows'.format((dt.datetime.now() - start).seconds, j*chunksize))

    df.to_sql('data', disk_engine, if_exists='append')
    index_start = df.index[-1] + 1
    if (j == 50): # only extract 1 million rows
        break
        
df = df.dropna() # clean up the data

9 seconds: completed 20000 rows
20 seconds: completed 40000 rows
31 seconds: completed 60000 rows
42 seconds: completed 80000 rows
54 seconds: completed 100000 rows
66 seconds: completed 120000 rows
78 seconds: completed 140000 rows
89 seconds: completed 160000 rows
101 seconds: completed 180000 rows
114 seconds: completed 200000 rows
125 seconds: completed 220000 rows
137 seconds: completed 240000 rows
149 seconds: completed 260000 rows
161 seconds: completed 280000 rows
172 seconds: completed 300000 rows
184 seconds: completed 320000 rows
197 seconds: completed 340000 rows
210 seconds: completed 360000 rows
222 seconds: completed 380000 rows
232 seconds: completed 400000 rows
243 seconds: completed 420000 rows
255 seconds: completed 440000 rows
266 seconds: completed 460000 rows
276 seconds: completed 480000 rows
289 seconds: completed 500000 rows
300 seconds: completed 520000 rows
312 seconds: completed 540000 rows
322 seconds: completed 560000 rows
332 seconds: completed 580000 row

## SQLAlchemy queries

In [7]:
# Import Table
from sqlalchemy import Table

engine = create_engine('sqlite:///311_NYC.db')

connection = engine.connect()

result = connection.execute("SELECT * FROM data LIMIT 3")

# Print table names
print(engine.table_names()) # only one table called data
print(result.fetchall())

['data']
[(1, '2011-06-20 10:53:34.000000', '2011-06-20 15:15:10.000000', 'DOF', 'DOF Property - Owner Issue', 'Remove Mortgage', 'GLEN OAKS'), (2, '2011-06-20 10:43:46.000000', '2011-06-30 11:33:32.000000', 'DOT', 'Street Condition', 'Defective Hardware', 'BROOKLYN'), (3, '2011-06-20 11:14:08.000000', '2011-06-21 18:41:35.000000', 'DHS', 'DHS Advantage - Tenant', 'Other Issue', 'SOUTH RICHMOND HILL')]


## Pandas Queries (easier & results look better)
Preview the data

In [8]:
df = pd.read_sql_query('SELECT * FROM data LIMIT 3', con=engine,index_col='index')
df.head()

,CreatedDate,ClosedDate,Agency,ComplaintType,Descriptor,City
index,,,,,,
1,2011-06-20 10:53:34.000000,2011-06-20 15:15:10.000000,DOF,DOF Property - Owner Issue,Remove Mortgage,GLEN OAKS
2,2011-06-20 10:43:46.000000,2011-06-30 11:33:32.000000,DOT,Street Condition,Defective Hardware,BROOKLYN
3,2011-06-20 11:14:08.000000,2011-06-21 18:41:35.000000,DHS,DHS Advantage - Tenant,Other Issue,SOUTH RICHMOND HILL


#### Select a few columns

In [10]:
df = pd.read_sql_query('SELECT Agency, Descriptor FROM data LIMIT 3', engine)
df.head()

,Agency,Descriptor
0,DOF,Remove Mortgage
1,DOT,Defective Hardware
2,DHS,Other Issue


#### Filter rows using WHERE

In [11]:
df = pd.read_sql_query('''SELECT ComplaintType, Descriptor, Agency
                          FROM data
                          WHERE Agency = "ACS"
                          LIMIT 10''', engine)
df.head()

,ComplaintType,Descriptor,Agency
0,Agency Issues,Family Court Issue,ACS
1,Agency Issues,Family Court Issue,ACS


#### Filter multiple values in a column using IN

In [20]:
df = pd.read_sql_query('SELECT ComplaintType, Descriptor, Agency '
                       'FROM data '
                       'WHERE Agency IN ("NYPD", "DOB")'
                       'LIMIT 20', engine)
df.head(20)

,ComplaintType,Descriptor,Agency
0,Noise - Street/Sidewalk,Loud Music/Party,NYPD
1,Blocked Driveway,No Access,NYPD
2,Blocked Driveway,No Access,NYPD
3,Blocked Driveway,Partial Access,NYPD
4,Noise - Residential,Loud Music/Party,NYPD
5,Derelict Vehicle,With License Plate,NYPD
6,Derelict Vehicle,With License Plate,NYPD
7,Blocked Driveway,No Access,NYPD
8,Noise - Park,Loud Music/Party,NYPD
9,Noise - Residential,Loud Music/Party,NYPD


#### Find Unique values using DISTINCT

In [15]:
df = pd.read_sql_query('SELECT DISTINCT Agency FROM data', engine)
df.head()

,Agency
0,DOF
1,DOT
2,DHS
3,DOHMH
4,NYPD


####  Query values using COUNT(*) and GROUP_BY

In [17]:
df = pd.read_sql_query('''SELECT Agency, COUNT(*) as num_complaints
                          FROM data 
                          GROUP BY Agency ''', engine)

df.head()

,Agency,num_complaints
0,3-1-1,1097
1,ACS,2
2,CAU,2
3,CHALL,16
4,COIB,1


#### Which Department receives the most complaints?  And which receives the least?

In [22]:
df = pd.read_sql_query('''SELECT Agency, COUNT(*) as num_complaints
                          FROM data 
                          GROUP BY Agency 
                          ORDER BY -num_complaints''', engine)
df.head()

,Agency,num_complaints
0,HPD,634543
1,NYPD,464918
2,DOT,234664
3,DSNY,111076
4,DOB,62095


#### What is the most common complaint type?

In [23]:
df = pd.read_sql_query('''SELECT ComplaintType, COUNT(*) as num_complaints, Agency
                          FROM data
                          GROUP BY ComplaintType
                          ORDER BY -num_complaints''', engine)

df.head()

,ComplaintType,num_complaints,Agency
0,HEAT/HOT WATER,309039,HPD
1,Noise - Residential,162078,NYPD
2,Blocked Driveway,93795,NYPD
3,Illegal Parking,86530,NYPD
4,Street Light Condition,78830,DOT


#### Number of complaints per city for the top 10 cities with most complaints

In [25]:
df = pd.read_sql_query('''SELECT City, COUNT(*) as num_complaints
                          FROM data
                          GROUP BY City
                          ORDER BY -num_complaints
                          LIMIT 10 ''', engine)
df

,City,num_complaints
0,BROOKLYN,512387
1,NEW YORK,370381
2,BRONX,349553
3,None,121667
4,STATEN ISLAND,70249
5,Jamaica,20384
6,JAMAICA,16440
7,Flushing,15542
8,FLUSHING,15390
9,ASTORIA,14157


#### Perform Case Insensitive Queries

In [26]:
df = pd.read_sql_query('''SELECT City, COUNT(*) as num_complaints
                          FROM data
                          GROUP BY City
                          COLLATE NOCASE
                          ORDER BY -num_complaints
                          LIMIT 11 ''', engine)
df

,City,num_complaints
0,BROOKLYN,512387
1,NEW YORK,370381
2,BRONX,349553
3,None,121667
4,STATEN ISLAND,70249
5,JAMAICA,36824
6,FLUSHING,30932
7,Astoria,27685
8,Ridgewood,18720
9,ELMHURST,14535
